In [1]:
from create_dataset import *
import pickle
import time
import numpy as np
from simplelmdb import Simplelmdb
import os

In [2]:
with open('data/labels/single_lane_recording_640_480.pkl') as f:
    source = pickle.load(f)
    


In [5]:
train_files = source['image_files'][:1500]
test_files = source['image_files'][1500:1750]
folder = 'data/datasets/single_lane_topdown_2_anchor/'
if not os.path.exists(folder):
    os.mkdir(folder)
targets_fn = calculate_2_anchor_displacements    
import time
start = time.time()
data_min, data_max = create_dataset(folder + 'train.lmdb', train_files, source, targets_fn, 2,size=(224,224), num_shifts=9, use_front=False, normalize_targets=True)
end = time.time()
print end-start

print "Creating Test"
start = time.time()
create_dataset(folder + 'test.lmdb', test_files,  source, targets_fn, 2, size=(224,224), num_shifts=9, data_min=data_min, data_max=data_max, use_front=False, normalize_targets=True) 
end = time.time()
print end-start

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
Normalizing
110.412045956
Creating Test
0
1000
2000
Normalizing
17.5124788284


In [6]:
a=Simplelmdb(folder+'/train.lmdb')

In [17]:
b=a.get('1235')

In [18]:
b['image_targets']

array([ 0.50483313,  0.52989558])